In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [2]:
# Model
model_name = "deepseek-ai/DeepSeek-R1-0528-Qwen3-8B"

In [3]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [4]:
# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)

config.json:   0%|          | 0.00/859 [00:00<?, ?B/s]

Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/7.77G [00:00<?, ?B/s]

model-00001-of-000002.safetensors:   0%|          | 0.00/8.61G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
# Prompt
prompt = "what is qbit."

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,   # how long the answer can be
        do_sample=True,       # randomness (for creative answers)
        temperature=0.7,      # lower = more factual, higher = more creative
        top_p=0.9
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Model output:\n", response)

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


Model output:
 what is qbit. and how does it relate to quantum computing? Qbit is the quantum version of a classical bit, but the classical bit can be 0 or 1, while a qbit can be 0 or 1 or both at the same time. That is the superposition. And also, the qbit is entangled with other qbits, which is another property. So, how does that relate to quantum computing?
A qbit, or quantum bit, is the fundamental unit of information in quantum computing, analogous to the classical bit in traditional computing. However, unlike a classical bit which can only exist in one of two definite states (0 or 1), a qbit can exist in a superposition of both states simultaneously. This means that while a classical bit is either 0 or 1, a qbit can be in a state that is a combination of 0 and 1, allowing it to represent multiple states at once.

Moreover, qbits can be entangled,


In [9]:
# Prompt
prompt = """
Let f(x) = ∫₀ˣ (sin(t²)/t) dt for x > 0.
1. Show that f(x) is well-defined for all x > 0.
2. Determine whether the improper integral ∫₀^∞ (sin(t²)/t) dt converges or diverges.
3. If it converges, approximate its value using the substitution u = t².
"""

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,   # how long the answer can be
        do_sample=True,       # randomness (for creative answers)
        temperature=0.7,      # lower = more factual, higher = more creative
        top_p=0.9
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Model output:\n", response)


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


Model output:
 
Let f(x) = ∫₀ˣ (sin(t²)/t) dt for x > 0. 
1. Show that f(x) is well-defined for all x > 0.
2. Determine whether the improper integral ∫₀^∞ (sin(t²)/t) dt converges or diverges.
3. If it converges, approximate its value using the substitution u = t².
To show that f(x) is well-defined, I need to show that the integral exists for all x > 0. Since it's an improper integral at t=0 because sin(t²)/t might blow up or something, but let's check the behavior at t=0.

When t=0, sin(t²)/t. As t approaches 0, t² approaches 0, sin(t²) approaches t², so sin(t²)/t approaches t²/t = t, which goes to 0. So the integrand is continuous at t=0 if we define it as 0 there, but actually sin(t²)/t at t=0 is undefined, but the limit exists and is 0, so we can define the function at 0 to be 0, and then the integral from 0 to x is proper for x>0, but still, since the upper limit is x, and x>0, but the integral is from


In [10]:
# Prompt with step-by-step instruction
prompt = """
Solve the following math problem step by step:

Let f(x) = ∫₀ˣ (sin(t²)/t) dt for x > 0.
1. Show that f(x) is well-defined for all x > 0.
2. Determine whether the improper integral ∫₀^∞ (sin(t²)/t) dt converges or diverges.
3. If it converges, approximate its value using the substitution u = t².
"""

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=400,   # more tokens for step-by-step reasoning
        do_sample=True,       # allows creative generation
        temperature=0.7,
        top_p=0.95,           # slightly higher to encourage diverse steps
        repetition_penalty=1.2,  # avoid repeating steps
        early_stopping=True
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Model output:\n", response)


The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


Model output:
 
Solve the following math problem step by step:

Let f(x) = ∫₀ˣ (sin(t²)/t) dt for x > 0. 
1. Show that f(x) is well-defined for all x > 0.
2. Determine whether the improper integral ∫₀^∞ (sin(t²)/t) dt converges or diverges.
3. If it converges, approximate its value using the substitution u = t².
4. Is there a function F such as lim_(x→∞) [f'(x)] / [F(π/x + c₁) - ln(a/b)x] exists and equals L? Find one.

I need to solve this problem: Let \( f(x) = \int_0^x \frac{\sin(t^2)}{t}   dt \) for x > 0.

First part: show that f(x) is well-defined for all x > 0.

Well-defined means we can actually compute these integrals without issues like division by zero or other discontinuities in an interval from 0 to any positive number less than infinity?

The issue here might be at t=0 because when you plug in t=0 into sin(t²)/t, let's see what happens.

At t=0, numerator sin(t²), since sine of anything near zero behaves linearly if I use Taylor series right off the bat...

Taylor expand 